In [40]:
! pip install nltk

# Training Word2vec (gensim)

In [146]:
import gensim
import pandas as pd

In [147]:
df = pd.read_csv("tweets-engagement-metrics.csv")
df.head()

,Unnamed: 0,UserID,Gender,LocationID,City,State,StateCode,Country,TweetID,Hour,Day,Weekday,IsReshare,Reach,RetweetCount,Likes,Klout,Sentiment,Lang,text
0,0,tw-1267804344,Unknown,1.0,Elbasan,Elbasan,AL,Albania,tw-698155297102295041,7,12,Friday,True,339.0,127.0,0.0,44.0,0.0,en,"""RT @AdrianRusso82: Our Innovation Lab is offi..."
1,1,tw-27229880,Male,2.0,Tirana,Tirane,AL,Albania,tw-685159757209059329,11,7,Thursday,False,87.0,0.0,0.0,22.0,0.0,en,Now Open AWS Asia Pacific (Seoul) Region via ...
2,2,tw-27229880,Male,2.0,Tirana,Tirane,AL,Albania,tw-686907710311378944,6,12,Tuesday,False,87.0,0.0,0.0,22.0,0.0,en,A Beginner's Guide to Scaling to 11 Million+ U...
3,3,tw-27229880,Male,2.0,Tirana,Tirane,AL,Albania,tw-686968158050201600,10,12,Tuesday,False,87.0,0.0,0.0,22.0,0.0,en,Bridging AWS and Azure environments via VPN vi...
4,4,tw-27229880,Male,2.0,Tirana,Tirane,AL,Albania,tw-690210449674092545,9,21,Thursday,False,85.0,0.0,0.0,21.0,0.0,en,ELK on AWS ElasticSearch Service + ElasticBean...


In [148]:
df.shape

(102062, 20)

In [149]:
review_text = df.text.apply(gensim.utils.simple_preprocess)

In [150]:
df.text[7]

'RT @pzerger: Microsoft and Google Lead in Cloud Growth AWS Rules Overall http://www.datamation.com/cloud-computing/microsoft-and-google-lead-in-cloud-growth-aws-rules-overall.html #Azure'

In [151]:
review_text.loc[7]

['rt',
 'pzerger',
 'microsoft',
 'and',
 'google',
 'lead',
 'in',
 'cloud',
 'growth',
 'aws',
 'rules',
 'overall',
 'http',
 'www',
 'datamation',
 'com',
 'cloud',
 'computing',
 'microsoft',
 'and',
 'google',
 'lead',
 'in',
 'cloud',
 'growth',
 'aws',
 'rules',
 'overall',
 'html',
 'azure']

In [165]:
model = gensim.models.Word2Vec(
    window=40,
    min_count=2,
    workers=4,
)

In [166]:
model.build_vocab(review_text, progress_per=1000)

In [167]:
model.train(review_text, total_examples=model.corpus_count, epochs=model.epochs)

(10028791, 13474175)

In [168]:
model.save("./Tweets.model")

In [169]:
model.wv.similarity(w1="great", w2="good")

0.69041467

In [170]:
model.wv.most_similar("bad")

[('yrprkv', 0.8794959783554077),
 ('addresses', 0.8272523880004883),
 ('ip', 0.7782436609268188),
 ('requests', 0.7722018361091614),
 ('generating', 0.7255535125732422),
 ('block', 0.7250475883483887),
 ('generate', 0.6471606492996216),
 ('input', 0.5709637403488159),
 ('compression', 0.5544778108596802),
 ('deleted', 0.5477204322814941)]

In [171]:
text1 = " how many seats are on the next flight"

In [172]:
text2 = "No warnings or communication until we were 15 minutes Late Flight."

In [173]:
text1 = pd.Series(text1).apply(gensim.utils.simple_preprocess)

In [174]:
text2 = pd.Series(text2).apply(gensim.utils.simple_preprocess)

In [175]:
text2.get(0)

['no',
 'warnings',
 'or',
 'communication',
 'until',
 'we',
 'were',
 'minutes',
 'late',
 'flight']

In [176]:
text1 = "Artificial intelligence (AI) is advancing at a rapid pace, revolutionizing various industries such as healthcare, finance, and transportation. Machine learning algorithms are enabling computers to perform tasks that previously required human intelligence, from diagnosing diseases to driving cars autonomously. The potential benefits of AI are immense, including improved efficiency, increased accuracy, and enhanced decision-making capabilities. However, there are also concerns about the ethical implications and potential risks associated with AI, such as job displacement, bias in algorithms, and loss of privacy. As AI continues to evolve, it is crucial for policymakers, researchers, and society as a whole to address these challenges and ensure that AI technologies are developed and deployed responsibly."
text2 = "The history of ancient civilizations is a fascinating subject that has captivated scholars and historians for centuries. From the majestic pyramids of Egypt to the intricate ruins of Machu Picchu, each civilization has left behind a legacy of art, architecture, and culture that continues to inspire awe and wonder today. By studying ancient texts, artifacts, and archaeological findings, researchers have been able to piece together the stories of these ancient societies and gain insights into their beliefs, customs, and daily lives. However, many aspects of ancient civilizations remain shrouded in mystery, leaving room for speculation and debate among experts. Despite the challenges of deciphering ancient languages and interpreting ancient sources, the study of ancient civilizations remains a vibrant and dynamic field of inquiry, offering valuable lessons about the human experience and our place in the world."

In [177]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import numpy as np

# Load pre-trained word2vec model
# word_vectors = KeyedVectors.load_word2vec_format('path_to_word2vec.bin', binary=True)

def get_sentence_vector(sentence, model):
    # Tokenize the sentence
    tokens = sentence.split()
    # Initialize an array to store word vectors
    word_vectors_list = []
    # Iterate over tokens and get word vectors
    for token in tokens:
        if token in model.wv:
            word_vectors_list.append(model.wv[token])
    # If no valid word vectors found, return None
    if not word_vectors_list:
        return None
    # Calculate the average of word vectors
    return np.mean(word_vectors_list, axis=0)

def cosine_similarity(vec1, vec2):
    # Compute cosine similarity between two vectors
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

# Example sentences
sentence1 = "the like dogs"
sentence2 = "Im Like dogs"

# Example word2vec model
# model = Word2Vec.load('path_to_word2vec_model')

# Get sentence vectors
vec1 = get_sentence_vector(text1, model)
vec2 = get_sentence_vector(text2, model)

if vec1 is not None and vec2 is not None:
    # Compute cosine similarity
    similarity = cosine_similarity(vec1, vec2)
    print("Similarity between the two sentences:", similarity)
else:
    print("Unable to compute similarity. Make sure the sentences contain valid words in the word2vec model vocabulary.")


Similarity between the two sentences: 0.8347887


# Pretrained Word2vec "GoogleNews" (gensim)

In [178]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MCC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [179]:
from gensim.models import KeyedVectors
from nltk.tokenize import word_tokenize
import numpy as np

# Load pre-trained Word2Vec model
# Load pre-trained Word2Vec model
model_path = "GoogleNews-vectors-negative300.bin.gz"
word_vectors = KeyedVectors.load_word2vec_format(model_path, binary=True)

# Example texts
text1 = "This is the first text."
text2 = "This is the second text."

# Tokenization and preprocessing
tokens1 = word_tokenize(text1.lower())
tokens2 = word_tokenize(text2.lower())

# Compute text embeddings
def text_embedding(tokens):
    embeddings = [word_vectors.get_vector(word) for word in tokens if word in word_vectors.key_to_index]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(word_vectors.vector_size)  # Return zero vector if no words found in vocabulary

embedding1 = text_embedding(tokens1)
embedding2 = text_embedding(tokens2)

# Calculate cosine similarity
similarity_score = np.dot(embedding1, embedding2) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))

print("Similarity Score:", similarity_score)


Similarity Score: 0.9780031


In [180]:
text1 = "Artificial intelligence (AI) is advancing at a rapid pace, revolutionizing various industries such as healthcare, finance, and transportation. Machine learning algorithms are enabling computers to perform tasks that previously required human intelligence, from diagnosing diseases to driving cars autonomously. The potential benefits of AI are immense, including improved efficiency, increased accuracy, and enhanced decision-making capabilities. However, there are also concerns about the ethical implications and potential risks associated with AI, such as job displacement, bias in algorithms, and loss of privacy. As AI continues to evolve, it is crucial for policymakers, researchers, and society as a whole to address these challenges and ensure that AI technologies are developed and deployed responsibly."
text2 = "The history of ancient civilizations is a fascinating subject that has captivated scholars and historians for centuries. From the majestic pyramids of Egypt to the intricate ruins of Machu Picchu, each civilization has left behind a legacy of art, architecture, and culture that continues to inspire awe and wonder today. By studying ancient texts, artifacts, and archaeological findings, researchers have been able to piece together the stories of these ancient societies and gain insights into their beliefs, customs, and daily lives. However, many aspects of ancient civilizations remain shrouded in mystery, leaving room for speculation and debate among experts. Despite the challenges of deciphering ancient languages and interpreting ancient sources, the study of ancient civilizations remains a vibrant and dynamic field of inquiry, offering valuable lessons about the human experience and our place in the world."
# Tokenization and preprocessing
tokens1 = word_tokenize(text1.lower())
tokens2 = word_tokenize(text2.lower())

# Compute text embeddings
def text_embedding(tokens):
    embeddings = [word_vectors.get_vector(word) for word in tokens if word in word_vectors.key_to_index]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(word_vectors.vector_size)  # Return zero vector if no words found in vocabulary

embedding1 = text_embedding(tokens1)
embedding2 = text_embedding(tokens2)

# Calculate cosine similarity
similarity_score = np.dot(embedding1, embedding2) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))

print("Similarity Score:", similarity_score)

Similarity Score: 0.6747796


In [181]:
text1 = """
"The sun rises in the east, casting its warm rays upon the earth. Birds chirp merrily as they greet the dawn, 
while the gentle breeze rustles through the leaves of the trees. It's a new day, full of promise and possibility."
"""

text2 =  """
"As the day begins, the sun ascends from the eastern horizon, illuminating the world with its radiant light. 
The melodious songs of birds fill the air, accompanied by the soft whispers of the wind among the branches. 
It's a fresh beginning, brimming with hope and opportunity."
"""
# Tokenization and preprocessing
tokens1 = word_tokenize(text1.lower())
tokens2 = word_tokenize(text2.lower())

# Compute text embeddings
def text_embedding(tokens):
    embeddings = [word_vectors.get_vector(word) for word in tokens if word in word_vectors.key_to_index]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(word_vectors.vector_size)  # Return zero vector if no words found in vocabulary

embedding1 = text_embedding(tokens1)
embedding2 = text_embedding(tokens2)

# Calculate cosine similarity
similarity_score = np.dot(embedding1, embedding2) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))

print("Similarity Score:", similarity_score)

Similarity Score: 0.8934646


In [182]:
text1 =  """
"The ocean waves crash against the rugged cliffs, sending salty spray into the air. Seagulls soar overhead, 
their cries echoing in the vast expanse of the sky. The scent of salt and seaweed permeates the air, a reminder 
of the eternal dance between land and sea."
"""

text2 = """
"On the coast, waves collide with the sturdy rocks, splashing droplets high into the air. Above, seagulls glide 
gracefully, their calls mingling with the sounds of the crashing waves. The briny aroma of the sea fills the 
atmosphere, a testament to the perpetual interplay of land and water."
"""
# Tokenization and preprocessing
tokens1 = word_tokenize(text1.lower())
tokens2 = word_tokenize(text2.lower())

# Compute text embeddings
def text_embedding(tokens):
    embeddings = [word_vectors.get_vector(word) for word in tokens if word in word_vectors.key_to_index]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(word_vectors.vector_size)  # Return zero vector if no words found in vocabulary

embedding1 = text_embedding(tokens1)
embedding2 = text_embedding(tokens2)

# Calculate cosine similarity
similarity_score = np.dot(embedding1, embedding2) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))

print("Similarity Score:", similarity_score)

Similarity Score: 0.8929715


In [183]:
text1 = """
"The city streets bustle with activity as people hurry to and fro, lost in their own thoughts and concerns. 
Car horns blare in a cacophony of sound, competing with the chatter of pedestrians and the hum of engines. 
Amidst the hustle and bustle, life carries on unabated."
"""

text2 = """
"In the quiet of the countryside, fields stretch out as far as the eye can see, bathed in the soft glow of 
the setting sun. The only sounds are the gentle rustle of the wind through the grass and the occasional chirp 
of a cricket. It's a tranquil scene, far removed from the chaos of the city."
"""
# Tokenization and preprocessing
tokens1 = word_tokenize(text1.lower())
tokens2 = word_tokenize(text2.lower())

# Compute text embeddings
def text_embedding(tokens):
    embeddings = [word_vectors.get_vector(word) for word in tokens if word in word_vectors.key_to_index]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(word_vectors.vector_size)  # Return zero vector if no words found in vocabulary

embedding1 = text_embedding(tokens1)
embedding2 = text_embedding(tokens2)

# Calculate cosine similarity
similarity_score = np.dot(embedding1, embedding2) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))

print("Similarity Score:", similarity_score)

Similarity Score: 0.7603578


In [184]:
text1 = "Artificial intelligence (AI) is advancing at a rapid pace, revolutionizing various industries such as healthcare, finance, and transportation. Machine learning algorithms are enabling computers to perform tasks that previously required human intelligence, from diagnosing diseases to driving cars autonomously. The potential benefits of AI are immense, including improved efficiency, increased accuracy, and enhanced decision-making capabilities. However, there are also concerns about the ethical implications and potential risks associated with AI, such as job displacement, bias in algorithms, and loss of privacy. As AI continues to evolve, it is crucial for policymakers, researchers, and society as a whole to address these challenges and ensure that AI technologies are developed and deployed responsibly."
text2 = """
"In the quiet of the countryside, fields stretch out as far as the eye can see, bathed in the soft glow of 
the setting sun. The only sounds are the gentle rustle of the wind through the grass and the occasional chirp 
of a cricket. It's a tranquil scene, far removed from the chaos of the city."
"""
# Tokenization and preprocessing
tokens1 = word_tokenize(text1.lower())
tokens2 = word_tokenize(text2.lower())

# Compute text embeddings
def text_embedding(tokens):
    embeddings = [word_vectors.get_vector(word) for word in tokens if word in word_vectors.key_to_index]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(word_vectors.vector_size)  # Return zero vector if no words found in vocabulary

embedding1 = text_embedding(tokens1)
embedding2 = text_embedding(tokens2)

# Calculate cosine similarity
similarity_score = np.dot(embedding1, embedding2) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))

print("Similarity Score:", similarity_score)

Similarity Score: 0.5747914


# Spacy

In [3]:
headlines = [
#Crypto
'Investors unfazed by correction as crypto funds see $154 million inflows',
'Bitcoin, Ethereum prices continue descent, but crypto funds see inflows',
 
#Inflation
'The surge in euro area inflation during the pandemic: transitory but with upside risks',
"Inflation: why it's temporary and raising interest rates will do more harm than good",
 
#common
'Will Cryptocurrency Protect Against Inflation?']

In [4]:
# !pip install -U spacy

In [5]:
! python -m spacy download en_core_web_md

     ---------------------------------------- 0.0/42.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/42.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/42.8 MB ? eta -:--:--
     --------------------------------------- 0.0/42.8 MB 326.8 kB/s eta 0:02:11
     --------------------------------------- 0.1/42.8 MB 521.8 kB/s eta 0:01:22
     --------------------------------------- 0.2/42.8 MB 871.5 kB/s eta 0:00:49
     ---------------------------------------- 0.3/42.8 MB 1.1 MB/s eta 0:00:39
     ---------------------------------------- 0.5/42.8 MB 1.6 MB/s eta 0:00:27
      --------------------------------------- 0.8/42.8 MB 2.2 MB/s eta 0:00:20
     - -------------------------------------- 1.1/42.8 MB 2.8 MB/s eta 0:00:16
     - -------------------------------------- 2.0/42.8 MB 4.5 MB/s eta 0:00:09
     -- ------------------------------------- 2.5/42.8 MB 5.1 MB/s eta 0:00:08
     -- ------------------------------------- 3.2/42.8 MB 5.9 MB/s eta

In [7]:
import spacy

In [8]:
nlp = spacy.load('en_core_web_md')
docs = [nlp(headline) for headline in headlines]

In [10]:
similarity = []
for i in range(len(docs)):
    row = []
    for j in range(len(docs)):
      row.append(docs[i].similarity(docs[j]))
    similarity.append(row)


NameError: name 'create_heatmap' is not defined

In [2]:
import spacy

# Load the SpaCy language model
nlp = spacy.load("en_core_web_md")

def preprocess_text(text):
    # Tokenize the text and remove stopwords
    doc = nlp(text)
    return [token.text.lower() for token in doc if not token.is_stop and not token.is_punct]

def calculate_similarity(text1, text2):
    # Preprocess texts
    tokens1 = preprocess_text(text1)
    tokens2 = preprocess_text(text2)
    
    # Calculate similarity
    similarity_sum = 0
    count = 0
    for token1 in tokens1:
        max_similarity = 0
        for token2 in tokens2:
            similarity = nlp(token1).similarity(nlp(token2))
            if similarity > max_similarity:
                max_similarity = similarity
        similarity_sum += max_similarity
        count += 1
    
    # Calculate average similarity
    if count > 0:
        avg_similarity = similarity_sum / count
    else:
        avg_similarity = 0
    
    return avg_similarity

# Example texts
text1 = "The city bustled with life, streets filled with the symphony of honking horns and hurried footsteps. Neon lights illuminated the night, casting colorful reflections on wet pavements. Cafés buzzed with conversation, and the aroma of coffee lingered in the air."
text2 = "The moon casts its soft glow over the silent night. Stars twinkle in the darkness, like diamonds scattered across velvet. The world sleeps peacefully, wrapped in a blanket of tranquility. A calmness settles in, soothing the restless soul with its gentle embrace."

# Calculate similarity
similarity_score = calculate_similarity(text1, text2)
print("Similarity Score:", similarity_score)


Similarity Score: 0.5029938457466879
